## Set up

### imports

In [9]:
import sys
import numpy as np
import keras
from sklearn.metrics import classification_report, accuracy_score
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard, CSVLogger
import pandas as pd
import gc
import json
import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
sys.path.append('../src')
from data.utils import load
from models.create_models import create_cnn_model, create_fcnn_model
from keras import backend as K


keras.utils.set_random_seed(42)

In [ ]:
tf.config.list_physical_devices('GPU')

### definitions

In [11]:
data_dir = "../data/"

# input image dimensions
img_rows, img_cols = 28, 28


PATH_MODELS = "../models/"
PATH_LOGS = "../logs/"
RESULTS_PATH = "../results/"

BATCH_SIZE = 128
EPOCHS = 10000

### Carregar dataset

In [ ]:
# Load the data
X_train = load(data_dir + 'kmnist-train-imgs.npz')
X_test = load(data_dir + 'kmnist-test-imgs.npz')
y_train = load(data_dir + 'kmnist-train-labels.npz')
y_test = load(data_dir + 'kmnist-test-labels.npz')


if K.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('{} train samples, {} test samples'.format(len(X_train), len(X_test)))
print(f'Input shape: {input_shape}')

# coleta dos valores unicos e das contagens
unique_train, counts_train = np.unique(y_train, return_counts=True)
unique_test, counts_test = np.unique(y_test, return_counts=True)

num_classes = len(unique_train) if list(unique_train) == list(unique_test) else None



# Convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

### Criar conjunto aumentado

In [5]:
# Crie o gerador de dados com data augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=False,
    fill_mode='nearest'
)

# Supomos que você tenha suas imagens de treino em x_train e labels em y_train
datagen.fit(X_train)

# Defina o número de amostras aumentadas que você deseja criar
num_augmented_samples = len(X_train) * 10  # por exemplo, 10 vezes o conjunto original

# Inicialize arrays para armazenar os dados aumentados
X_train_augmented = np.zeros((num_augmented_samples, *X_train.shape[1:]), dtype=np.float32)
y_train_augmented = np.zeros((num_augmented_samples, *y_train.shape[1:]), dtype=np.float32)

# Gere os dados aumentados
i = 0
for x_batch, y_batch in datagen.flow(X_train, y_train, batch_size=BATCH_SIZE):
    if i >= num_augmented_samples:
        break
    batch_size_aux = x_batch.shape[0]
    X_train_augmented[i:i+batch_size_aux] = x_batch
    y_train_augmented[i:i+batch_size_aux] = y_batch
    i += batch_size_aux

## Analysis

In [ ]:
print("Num class train: ", len(unique_train))
print("Num class test: ", len(unique_test))


print('### Treino ###')
for i in range(len(unique_train)): print(f'classe {unique_train[i]}: {counts_train[i]} observações')
print('\n### Teste ###')
for i in range(len(unique_test)): print(f'classe {unique_test[i]}: {counts_test[i]} observações')

## Classification

In [ ]:
METRICS =[
        'accuracy',
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name='auc'),
        keras.metrics.TruePositives(name='tp'),
        keras.metrics.TrueNegatives(name='tn'),
        keras.metrics.FalsePositives(name='fp'),
        keras.metrics.FalseNegatives(name='fn')
    ]

### CNN Benchmark

#### Normal

In [ ]:
# callbacks do modelo benchmark
callbacks_benchmark = [
    ModelCheckpoint(PATH_MODELS + f'best_model_CNN_benchmark.keras', save_best_only=True, monitor='val_loss'),
    TensorBoard(log_dir=PATH_LOGS),
    CSVLogger(PATH_MODELS + f'training_log_CNN_benchmark.csv')
]

# limpa sessao do keras
keras.backend.clear_session()
gc.collect()

# cria modelo
model_benchmark = create_cnn_model(
    input_shape=input_shape,
    num_classes= num_classes,
    metrics= METRICS
)

# treina modelo
history_benchmark = model_benchmark.fit(X_train, y_train,
          batch_size=BATCH_SIZE,
          epochs=12,
          verbose=1,
          validation_data=(X_test, y_test),
          callbacks = callbacks_benchmark
          )

# realiza predicao
train_pred_proba = model_benchmark.predict(X_train)
test_pred_proba = model_benchmark.predict(X_test)

# salva predicao no treino
test_results_benchmark = pd.DataFrame({
    "y_test": y_test.argmax(axis=1),
    "y_test_pred": np.argmax(test_pred_proba, axis =1),
    "y_test_pred_proba": list(test_pred_proba)
    
})
test_results_benchmark.to_csv(RESULTS_PATH + 'test/cnn_benchmark_test_results.csv', index = False)

# salva predicao no teste
train_results_benchmark = pd.DataFrame({
    "y_train": y_train.argmax(axis=1),
    "y_train_pred": np.argmax(train_pred_proba, axis =1),
    "y_train_pred_proba": list(train_pred_proba)
})
train_results_benchmark.to_csv(RESULTS_PATH + 'train/cnn_benchmark_train_results.csv', index = False)

json.dump(history_benchmark.history, open(RESULTS_PATH + "history/benchmark_cnn_history.json", "w"))


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Extraindo os pesos da primeira camada convolucional
filters, biases = model_benchmark.layers[0].get_weights()
n_filters = filters.shape[-1]

# Normalizando os filtros para a faixa [0, 1]
filters_min = filters.min()
filters_max = filters.max()
filters = (filters - filters_min) / (filters_max - filters_min)

# Visualizando os filtros
fig, axs = plt.subplots(1, n_filters, figsize=(20, 20))
for i in range(n_filters):
    f = filters[:, :, :, i]
    axs[i].imshow(f[:, :, 0], cmap='gray')
    axs[i].axis('off')

plt.show()


#### Com Data Augmentation

In [ ]:
# callbacks do modelo benchmark
callbacks_benchmark_aug = [
    ModelCheckpoint(PATH_MODELS + f'best_model_CNN_aug_benchmark.keras', save_best_only=True, monitor='val_loss'),
    TensorBoard(log_dir=PATH_LOGS),
    CSVLogger(PATH_MODELS + f'training_log_CNN_aug_benchmark.csv')
]

# limpa sessao do keras
keras.backend.clear_session()
gc.collect()

# cria modelo
model_benchmark_aug = create_cnn_model(
    input_shape=input_shape,
    num_classes= num_classes,
    metrics= METRICS
)

# treina modelo
history_benchmark_aug = model_benchmark_aug.fit(X_train_augmented, y_train_augmented,
          batch_size=BATCH_SIZE,
          epochs=12,
          verbose=1,
          validation_data=(X_test, y_test),
          callbacks = callbacks_benchmark_aug
          )

# realiza predicao
train_pred_proba = model_benchmark_aug.predict(X_train)
test_pred_proba = model_benchmark_aug.predict(X_test)

# salva predicao no treino
test_results_benchmark_aug = pd.DataFrame({
    "y_test": y_test.argmax(axis=1),
    "y_test_pred": np.argmax(test_pred_proba, axis =1),
    "y_test_pred_proba": list(test_pred_proba)
    
})
test_results_benchmark_aug.to_csv(RESULTS_PATH + 'test/cnn_benchmark_aug_test_results.csv', index = False)

# salva predicao no teste
train_results_benchmark_aug = pd.DataFrame({
    "y_train": y_train.argmax(axis=1),
    "y_train_pred": np.argmax(train_pred_proba, axis =1),
    "y_train_pred_proba": list(train_pred_proba)
})
train_results_benchmark_aug.to_csv(RESULTS_PATH + 'train/cnn_benchmark_aug_train_results.csv', index = False)

json.dump(history_benchmark_aug.history, open(RESULTS_PATH + "history/benchmark_aug_cnn_history.json", "w"))


### CNN

#### Normal

In [ ]:
# callbacks cnn
callbacks_cnn = [
    EarlyStopping(monitor='val_loss', patience=10),
    ModelCheckpoint(PATH_MODELS + f'best_model_CNN.keras', save_best_only=True, monitor='val_loss'),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5),
    TensorBoard(log_dir=PATH_LOGS),
    CSVLogger(PATH_MODELS + f'training_log_CNN.csv')
]

# limpa sessao do keras
keras.backend.clear_session()
gc.collect()

# cria modelo
model_cnn = create_cnn_model(
    input_shape=input_shape,
    num_classes= num_classes,
    metrics = METRICS
)

# treina modelo
history_cnn = model_cnn.fit(X_train, y_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          verbose=1,
          validation_data=(X_test, y_test),
          callbacks=callbacks_cnn
          )


# realiza predicao
train_pred_proba = model_cnn.predict(X_train)
test_pred_proba = model_cnn.predict(X_test)

# salva predicao no treino
test_results_cnn = pd.DataFrame({
    "y_test": y_test.argmax(axis=1),
    "y_test_pred": np.argmax(test_pred_proba, axis =1),
    "y_test_pred_proba": list(test_pred_proba)
    
})
test_results_cnn.to_csv(RESULTS_PATH + 'test/cnn_test_results.csv', index = False)

# salva predicao no teste
train_results_cnn = pd.DataFrame({
    "y_train": y_train.argmax(axis=1),
    "y_train_pred": np.argmax(train_pred_proba, axis =1),
    "y_train_pred_proba": list(train_pred_proba)
})
train_results_cnn.to_csv(RESULTS_PATH + 'train/cnn_train_results.csv', index = False)


json.dump(history_cnn.history, open(RESULTS_PATH + "history/cnn_history.json", "w"))

#### Sem pooling

In [ ]:
# callbacks cnn
callbacks_cnn_wo_poll = [
    EarlyStopping(monitor='val_loss', patience=10),
    ModelCheckpoint(PATH_MODELS + f'best_model_CNN_without_polling.keras', save_best_only=True, monitor='val_loss'),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5),
    TensorBoard(log_dir=PATH_LOGS),
    CSVLogger(PATH_MODELS + f'training_log_CNN_without_polling.csv')
]

# limpa sessao do keras
keras.backend.clear_session()
gc.collect()

# cria modelo
model_cnn_wo_poll = create_cnn_model(
    input_shape=input_shape,
    num_classes= num_classes,
    metrics = METRICS,
    polling = False
)

# treina modelo
history_cnn_wo_poll = model_cnn_wo_poll.fit(X_train, y_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          verbose=1,
          validation_data=(X_test, y_test),
          callbacks=callbacks_cnn_wo_poll
          )


# realiza predicao
train_pred_proba = model_cnn_wo_poll.predict(X_train)
test_pred_proba = model_cnn_wo_poll.predict(X_test)

# salva predicao no treino
test_results_cnn_wo_poll = pd.DataFrame({
    "y_test": y_test.argmax(axis=1),
    "y_test_pred": np.argmax(test_pred_proba, axis =1),
    "y_test_pred_proba": list(test_pred_proba)
    
})
test_results_cnn_wo_poll.to_csv(RESULTS_PATH + 'test/cnn_without_polling_test_results.csv', index = False)

# salva predicao no teste
train_results_cnn_wo_poll = pd.DataFrame({
    "y_train": y_train.argmax(axis=1),
    "y_train_pred": np.argmax(train_pred_proba, axis =1),
    "y_train_pred_proba": list(train_pred_proba)
})
train_results_cnn_wo_poll.to_csv(RESULTS_PATH + 'train/cnn_without_polling_train_results.csv', index = False)


json.dump(history_cnn_wo_poll.history, open(RESULTS_PATH + "history/cnn_without_polling_history.json", "w"))

#### Com data augmentation

In [ ]:
# callbacks cnn
callbacks_cnn_aug = [
    EarlyStopping(monitor='val_loss', patience=10),
    ModelCheckpoint(PATH_MODELS + f'best_model_CNN_aug.keras', save_best_only=True, monitor='val_loss'),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5),
    TensorBoard(log_dir=PATH_LOGS),
    CSVLogger(PATH_MODELS + f'training_log_CNN_aug.csv')
]

# limpa sessao do keras
keras.backend.clear_session()
gc.collect()

# cria modelo
model_cnn_aug = create_cnn_model(
    input_shape=input_shape,
    num_classes= num_classes,
    metrics = METRICS
)

# treina modelo
history_cnn_aug = model_cnn_aug.fit(X_train_augmented, y_train_augmented,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          verbose=1,
          validation_data=(X_test, y_test),
          callbacks=callbacks_cnn_aug
          )


# realiza predicao
train_pred_proba = model_cnn_aug.predict(X_train)
test_pred_proba = model_cnn_aug.predict(X_test)

# salva predicao no treino
test_results_cnn_aug = pd.DataFrame({
    "y_test": y_test.argmax(axis=1),
    "y_test_pred": np.argmax(test_pred_proba, axis =1),
    "y_test_pred_proba": list(test_pred_proba)
    
})
test_results_cnn_aug.to_csv(RESULTS_PATH + 'test/cnn_aug_test_results.csv', index = False)

# salva predicao no teste
train_results_cnn_aug = pd.DataFrame({
    "y_train": y_train.argmax(axis=1),
    "y_train_pred": np.argmax(train_pred_proba, axis =1),
    "y_train_pred_proba": list(train_pred_proba)
})
train_results_cnn_aug.to_csv(RESULTS_PATH + 'train/cnn_aug_train_results.csv', index = False)


json.dump(history_cnn_aug.history, open(RESULTS_PATH + "history/cnn_aug_history.json", "w"))

### Fully Connected

#### Normal

In [ ]:
# callbacks cnn
callbacks_fcnn = [
    EarlyStopping(monitor='val_loss', patience=10),
    ModelCheckpoint(PATH_MODELS + f'best_model_fcnn.keras', save_best_only=True, monitor='val_loss'),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5),
    TensorBoard(log_dir=PATH_LOGS),
    CSVLogger(PATH_MODELS + f'training_log_fcnn.csv')
]

# limpa sessao do keras
keras.backend.clear_session()

# cria modelo
model_fcnn = create_fcnn_model(
    input_shape=input_shape,
    num_classes= num_classes,
    metrics = METRICS
)

# treina modelo
history_fcnn = model_fcnn.fit(X_train, y_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          verbose=1,
          validation_data=(X_test, y_test),
          callbacks=callbacks_fcnn
          )


# realiza predicao
train_pred_proba = model_fcnn.predict(X_train)
test_pred_proba = model_fcnn.predict(X_test)

# salva predicao no treino
test_results_fcnn = pd.DataFrame({
    "y_test": y_test.argmax(axis=1),
    "y_test_pred": np.argmax(test_pred_proba, axis =1),
    "y_test_pred_proba": list(test_pred_proba)
    
})
test_results_fcnn.to_csv(RESULTS_PATH + 'test/fcnn_test_results.csv', index = False)


# salva predicao no teste
train_results_fcnn = pd.DataFrame({
    "y_train": y_train.argmax(axis=1),
    "y_train_pred": np.argmax(train_pred_proba, axis =1),
    "y_train_pred_proba": list(train_pred_proba)
})
train_results_fcnn.to_csv(RESULTS_PATH + 'train/fcnn_train_results.csv', index = False)


json.dump(history_fcnn.history, open(RESULTS_PATH + "history/fcnn_history.json", "w"))

#### com data augmentation

In [ ]:
# callbacks cnn
callbacks_fcnn_aug = [
    EarlyStopping(monitor='val_loss', patience=10),
    ModelCheckpoint(PATH_MODELS + f'best_model_fcnn_aug.keras', save_best_only=True, monitor='val_loss'),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5),
    TensorBoard(log_dir=PATH_LOGS),
    CSVLogger(PATH_MODELS + f'training_log_fcnn_aug.csv')
]

# limpa sessao do keras
keras.backend.clear_session()

# cria modelo
model_fcnn_aug = create_fcnn_model(
    input_shape=input_shape,
    num_classes= num_classes,
    metrics = METRICS
)

# treina modelo
history_fcnn_aug = model_fcnn_aug.fit(X_train_augmented, y_train_augmented,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          verbose=1,
          validation_data=(X_test, y_test),
          callbacks=callbacks_fcnn_aug
          )


# realiza predicao
train_pred_proba = model_fcnn_aug.predict(X_train)
test_pred_proba = model_fcnn_aug.predict(X_test)

# salva predicao no treino
test_results_fcnn_aug = pd.DataFrame({
    "y_test": y_test.argmax(axis=1),
    "y_test_pred": np.argmax(test_pred_proba, axis =1),
    "y_test_pred_proba": list(test_pred_proba)
    
})
test_results_fcnn_aug.to_csv(RESULTS_PATH + 'test/fcnn_aug_test_results.csv', index = False)


# salva predicao no teste
train_results_fcnn_aug = pd.DataFrame({
    "y_train": y_train.argmax(axis=1),
    "y_train_pred": np.argmax(train_pred_proba, axis =1),
    "y_train_pred_proba": list(train_pred_proba)
})
train_results_fcnn_aug.to_csv(RESULTS_PATH + 'train/fcnn_aug_train_results.csv', index = False)


json.dump(history_fcnn_aug.history, open(RESULTS_PATH + "history/fcnn_aug_history.json", "w"))